In [1]:
# imports
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import urllib.request
from PIL import Image

In [2]:
# site request
url = "https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find_all("table", class_="roundy")

In [3]:
# get html of site
results = str(results)
results = results.split("<")

# create lists for types and image locations
tags = [i for i in results if i.startswith('a href=')]
images = [i for i in results if i.startswith('img alt=')]
image_list = []
for i in range(len(images)):
    image_list.append(images[i].split("src=")[1].split()[3][2:])

In [4]:
# create dataframe with information
pokedex_2 = pd.DataFrame()
pokedex_2["image_address"] = image_list

# add columns
pokedex_2["image_name"] = pokedex_2["image_address"].apply(lambda x: x.split("/")[-1] ).str.replace('^140px-', '')
pokedex_2["image_name"] = pokedex_2["image_name"].str.replace('\d+', '')
pokedex_2["image_name"] = "pokedex2_" + pokedex_2["image_name"]
pokedex_2["name"] = pokedex_2["image_name"].apply(lambda x: x.replace(".png", ""))
pokedex_2["name"] = pokedex_2["name"].apply(lambda x: x.replace("pokedex2_", ""))

/var/folders/x7/5j65n6x57hq9g5yst5z9k1m00000gn/T/ipykernel_3096/793527063.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  pokedex_2["image_name"] = pokedex_2["image_address"].apply(lambda x: x.split("/")[-1] ).str.replace('^140px-', '')
/var/folders/x7/5j65n6x57hq9g5yst5z9k1m00000gn/T/ipykernel_3096/793527063.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  pokedex_2["image_name"] = pokedex_2["image_name"].str.replace('\d+', '')


In [5]:
pokedex_2

,image_address,image_name,name
0,archives.bulbagarden.net/media/upload/thumb/f/...,pokedex2_Bulbasaur.png,Bulbasaur
1,archives.bulbagarden.net/media/upload/thumb/8/...,pokedex2_Ivysaur.png,Ivysaur
2,archives.bulbagarden.net/media/upload/thumb/6/...,pokedex2_Venusaur.png,Venusaur
3,archives.bulbagarden.net/media/upload/thumb/2/...,pokedex2_Charmander.png,Charmander
4,archives.bulbagarden.net/media/upload/thumb/0/...,pokedex2_Charmeleon.png,Charmeleon
...,...,...,...
1155,archives.bulbagarden.net/media/upload/thumb/d/...,pokedex2_Ogerpon.png,Ogerpon
1156,archives.bulbagarden.net/media/upload/thumb/2/...,pokedex2_Okidogi.png,Okidogi
1157,archives.bulbagarden.net/media/upload/thumb/5/...,pokedex2_Munkidori.png,Munkidori
1158,archives.bulbagarden.net/media/upload/thumb/6/...,pokedex2_Fezandipiti.png,Fezandipiti


In [6]:
# preparing tags list
tags_string = " ".join(tags).split("(Pok%C3%A9mon)")[::2]

# cleaning up information
poke_list2 = []
for i in range(1, (len(tags_string) - 5)):
    dict_i = tags_string[i].split(">")
    poke_list2.append(dict_i)

for i in range(len(poke_list2)):
    poke_list2[i].pop()

pokedex_2 = pokedex_2[:-5]

poke_list3 = []
for i in range(len(poke_list2)):
    name2 = poke_list2[i][1].split(" a href=")[0]
    type1 = poke_list2[i][1].split('"')[-2].replace(" (type)", "")
    if len(poke_list2[i]) == 3:
        type2 = poke_list2[i][2].split('"')[-2].replace(" (type)", "")
    else:
        type2 = np.nan
    key_names = ["name2", "type1", "type2"]
    values = [name2, type1, type2]
    dict_i = dict(zip(key_names, values))
    poke_list3.append(dict_i)

In [7]:
# combine dataframes
pokedex3 = pd.DataFrame(poke_list3)
pokedex3 = pokedex3.drop(columns = "name2")
pokedex4 = pd.merge(pokedex_2, pokedex3, left_index=True, right_index=True)
pokedex4.head()

,image_address,image_name,name,type1,type2
0,archives.bulbagarden.net/media/upload/thumb/f/...,pokedex2_Bulbasaur.png,Bulbasaur,Grass,Poison
1,archives.bulbagarden.net/media/upload/thumb/8/...,pokedex2_Ivysaur.png,Ivysaur,Grass,Poison
2,archives.bulbagarden.net/media/upload/thumb/6/...,pokedex2_Venusaur.png,Venusaur,Grass,Poison
3,archives.bulbagarden.net/media/upload/thumb/2/...,pokedex2_Charmander.png,Charmander,Fire,NaN
4,archives.bulbagarden.net/media/upload/thumb/0/...,pokedex2_Charmeleon.png,Charmeleon,Fire,NaN


In [8]:
# rename dataframe
pokedex2 = pokedex4

In [9]:
# download images locally

for index, row in pokedex2.iterrows():
    url = row["image_address"]
    name = row["image_name"]
    response = requests.get(f'https://{url}')
    path = (f"data/images/{name}")
    with open (path, "wb") as file:
        file.write(response.content)

In [10]:
# store dataframe as csv locally
from pathlib import Path  
filepath = Path('data/pokedex2.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
pokedex4.to_csv(filepath) 